In [1]:
import warnings
warnings.filterwarnings('ignore')

# A very simple power system model tutorial

Based on an open-source set of test models available at [github.com/ahilbers/renewable_test_PSMs](https://github.com/ahilbers/renewable_test_PSMs).

The model in this tutorial is very simple: it has one region, in which demand must be met by four generation technologies: baseload (cf. nuclear), peaking (cf. gas), wind, and solar.

In [2]:
import models

## Import time series data

In [3]:
ts_data = models.load_time_series_data('1_region')
ts_data = ts_data.loc['2017-06-08':'2017-06-15']
ts_data.head(10)

,demand,wind,solar
t,,,
2017-06-08 00:00:00,28.38,0.3456,0.000000
2017-06-08 01:00:00,27.04,0.3624,0.000000
2017-06-08 02:00:00,26.13,0.3609,0.000000
2017-06-08 03:00:00,25.61,0.3572,0.000095
2017-06-08 04:00:00,26.06,0.3480,0.012418
2017-06-08 05:00:00,30.16,0.3624,0.042604
2017-06-08 06:00:00,35.57,0.4005,0.078995
2017-06-08 07:00:00,39.23,0.4608,0.133953
2017-06-08 08:00:00,41.01,0.5247,0.191949


## `Operate` mode: meet demand with a fixed system

In this example, the generation capacities are user-defined, and the model optimises how demand is met. This is sometimes called the *economic dispatch* or the *unit commitment* problem.

In [4]:
# Set the generation capacities
generation_capacities = {'cap_baseload_total': 15.0,
                         'cap_peaking_total': 10.0,
                         'cap_wind_total': 20.0,
                         'cap_solar_total': 15.0}  # GW

# Create the model with the fixed capacities
model = models.OneRegionModel(ts_data=ts_data,
                              run_mode='operate',
                              fixed_caps=generation_capacities)

In [5]:
# Run the model, solve the optimisation problem
model.run()

In [6]:
# Show a summary of outputs
model.get_summary_outputs()

,output
cap_baseload_total,1.500000e+01
cap_peaking_total,1.000000e+01
cap_wind_total,2.000000e+01
cap_solar_total,1.500000e+01
peak_unmet_total,9.264780e+00
gen_baseload_total,1.205245e+05
gen_peaking_total,3.905862e+04
gen_wind_total,1.008980e+05
gen_solar_total,3.306704e+04
gen_unmet_total,7.521268e+03


In [ ]:
# Change colors (for the plot only) 
model._model_data.colors.loc['peaking'] = '#888888'
model._model_data.colors.loc['baseload'] = '#003399'
model._model_data.colors.loc['solar'] = '#e3bb12'
model._model_data.colors.loc['demand_power'] = '#000000'
model._model_data.colors.loc['unmet'] = '#000000'
model._model_data.colors.loc['wind'] = '#4eba44'

# Note: these plots do not display well in Github -- use your own notebook
model.plot.timeseries(array='all')

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_operate')
model.to_csv('outputs_operate')

## `Plan` mode: design cost-optimal system

In this example, the model optimises both the design and operation of a system. This is sometimes called the *capacity expansion planning* problem.

In [8]:
# Create the model without any fixed capacities
model = models.OneRegionModel(ts_data=ts_data,
                              run_mode='plan')

In [9]:
# Run the model, solve the optimisation problem
model.run()

In [10]:
# Show a summary of outputs
model.get_summary_outputs()

,output
cap_baseload_total,0.000000e+00
cap_peaking_total,3.263996e+01
cap_wind_total,3.268362e+01
cap_solar_total,3.651193e+01
peak_unmet_total,0.000000e+00
gen_baseload_total,0.000000e+00
gen_peaking_total,7.961009e+04
gen_wind_total,1.542031e+05
gen_solar_total,6.725612e+04
gen_unmet_total,0.000000e+00


In [ ]:
# Change colors (for the plot only) 
model._model_data.colors.loc['peaking'] = '#888888'
model._model_data.colors.loc['baseload'] = '#003399'
model._model_data.colors.loc['solar'] = '#e3bb12'
model._model_data.colors.loc['demand_power'] = '#000000'
model._model_data.colors.loc['unmet'] = '#000000'
model._model_data.colors.loc['wind'] = '#4eba44'

# Note: these plots do not display well in Github -- use your own notebook
model.plot.timeseries(array='all')

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_plan')
model.to_csv('outputs_plan')